<a href="https://colab.research.google.com/github/ajitaravind/chat-with-a-camera-manual/blob/main/Chat_with_camera_manual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In the notebook, we will build an application to chat with a Nikon Z8 camera manual. This could be used for any pdf documents (car manual, phone manual, you name it.), just replace the manuals with your own files.

Let's install all necessary packages

In [2]:
%%capture
pip install langchain langchain_community langchain_core langchain_groq langchain_nomic chromadb

Read the api keys from 'Secrets'

In [3]:
import os
from google.colab import userdata

os.environ["NOMIC_API_KEY"] = userdata.get('NOMIC_API_KEY')
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

Import all the necessary packages from langchain. We will use the excellent embedding function from Nomic and for vector store, we will use Chroma. PyPDF is our pdf loader

In [4]:
from langchain_community.document_loaders import PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_nomic.embeddings import NomicEmbeddings

In [5]:
embeddings = NomicEmbeddings(model="nomic-embed-text-v1.5")

In [8]:
%%capture
pip install pypdf

Loading and splitting documents to store into our Vector database.

In [9]:
loaders = [PyPDFLoader("Z9Z8_TG_Wildlife_(En)01.pdf"),
           PyPDFLoader("Z8UMEUR_(En)03.pdf")]

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=200
    )
docs = []
for loader in loaders:
        docs.extend(loader.load())
model_splits = text_splitter.split_documents(docs)

We got 156 chunks that get loaded to Chroma DB

In [11]:
len(model_splits)

156

In [12]:
vectorstore = Chroma.from_documents(model_splits, embeddings,persist_directory = "mydb")

In [13]:
retriever = vectorstore .as_retriever(search_type = "similarity")

Let's test our retriever to see if it's working fine

In [14]:
retriever.get_relevant_documents("What lens options I have above 600mm focal length?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='Shooting conditions:\nShutter speed: ¹/₅₀₀ s\nAperture: f/4\nFocal length: 600 mm\nISO sensitivity: ISO 1600NIKKOR Z 600mm f/4 TC VR S\nA 600 mm lens with a built-in 1.4× teleconverter allows you to quickly change focal lengths and\ncapture the skin texture of wildlife with incredible realism.\n8 Things to Bring', metadata={'page': 7, 'source': 'Z9Z8_TG_Wildlife_(En)01.pdf'}),
 Document(page_content='© Ramesh Karmakar\nShooting conditions:\nShutter speed: ¹/₃₂₀₀ s\nAperture: f/6.3\nFocal length: 600 mm (equivalent to 900 mm in 35 mm format with DX crop)\nISO sensitivity: ISO 2000NIKKOR Z 600mm f/6.3 VR S\nA 600 mm lens, which has been significantly reduced in size and weight by adopting a Phase Fresnel\n(PF) element, allows you to easily track a wild bird in flight handheld.\n9 Things to Bring', metadata={'page': 8, 'source': 'Z9Z8_TG_Wildlife_(En)01.pdf'}),
 Document(page_content='© Natsumi Handa\nShooting conditions:\nShutter speed: ¹/₅₀₀ s\nAperture: f/5.6\nF

Now let's create our LCEL chain to build the QA functionality

In [22]:
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough,RunnableParallel

Define the prompt and prompt template

In [15]:
qa_template = """
{question}
You are a helpful assistant from Nkon specialized in answering camera related queries. Use the following pieces of retrieved context to answer the question.\
If you don't know the answer, just say that you don't know.
Context:
{context}
"""

In [17]:
QA_CHAIN_PROMPT = PromptTemplate.from_template(qa_template)

We use the excellent llama3 as our LLM, ChatGroq helps us with super fast inference speed. Langchain makes it easier to integrate Groq.

In [19]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

Below is our chain. Prompt expects a dictionary with question and context (documents that we chunked and embedded). We pass along the question as it is to prompt with help of RunnablePassthrough and for context we invoke the retriever to get relevant documents from our vector store.

In [23]:
chain = RunnableParallel(
        {"question" : RunnablePassthrough(),
        "context" : retriever
        }
        ) |QA_CHAIN_PROMPT| llm | StrOutputParser()

We are all set! Let's run the chain

In [24]:
result = chain.invoke("Please suggest few lenses for widlife photography that have atleast 400mm focal length")

Voila, here we go. Pretty cool isn't it.

In [25]:
result

"Wildlife photography! I'd be happy to help you with some fantastic lens options that have a minimum focal length of 400mm.\n\nBased on the provided context, here are a few lenses that fit your requirement:\n\n1. **NIKKOR Z 400mm f/4.5 VR S**: A compact and lightweight lens that allows for handheld shooting, perfect for capturing birds in flight or in vast landscapes.\n2. **NIKKOR Z 400mm f/2.8 TC VR S**: This lens features a built-in 1.4x teleconverter, allowing you to quickly switch between focal lengths to suit your scene. Ideal for capturing vivid images of small wild birds in motion.\n3. **NIKKOR Z 100-400mm f/4.5-5.6 VR S**: A versatile zoom lens that offers high optical performance, making it perfect for capturing moving animals from a distance. The smooth zooming and animal-detection AF features make it an excellent choice.\n4. **NIKKOR Z 800mm f/6.3 VR S**: If you need even more reach, this lens is a great option. Its Phase Fresnel (PF) element design reduces the size and weig

Let's ask one more question

In [26]:
result = chain.invoke("Can you give some tips for photographing birds in flight?")

That's perfect, bingo!

In [27]:
result

"Photographing birds in flight! That's a thrilling challenge. As a helpful assistant from Nikon, I'd be delighted to share some tips to help you capture stunning images of birds in flight:\n\n1. **Choose the right gear**: A super-telephoto lens (at least 200mm) is essential for capturing birds in flight. A camera with good autofocus and burst mode capabilities is also crucial.\n2. **Select the right settings**: Use a fast shutter speed (at least 1/2000th of a second) to freeze the motion of the bird. Set your camera to Continuous High-Speed mode (up to 20 fps) to increase your chances of capturing the perfect shot.\n3. **Aperture and ISO**: Use the maximum aperture value of your lens or stop down by 1/2 EV to ensure a sufficient depth of field. Set your ISO to Auto, with a maximum sensitivity of 2000.\n4. **Autofocus mode**: Use [Auto-area AF] or [3D-tracking] mode to help track the bird's movement. You can also assign [3D-tracking] to a Fn button using Custom Setting f2 for quick acce